# Documentation: Data Processing and Transformation Notebook
Overview

This notebook provides a structured workflow for pre-processing data before loading it into SAP HANA Cloud. It automates the conversion, transformation, and preparation of data files into a format suitable for database ingestion.

Key Features

Excel to CSV Conversion

CSV Processing

Cleans and standardizes CSV content.

Handles formatting issues (e.g., missing values, data type corrections, column name sanitization).

Data Transformation

Applies necessary transformations to align with SAP HANA Cloud schema requirements.

Supports field renaming, type casting, and normalization steps.

SQL Script Generation

For each sheet in the original Excel file, the notebook generates:

A CREATE TABLE SQL command.

Schema definitions mapped from the transformed CSVs.

Ensures that table creation scripts are consistent with HANA Cloud standards.

# Excel to CSV Conversion

Converts uploaded Excel spreadsheets into multiple CSV files (one per sheet).

In [1]:
import pandas as pd
import os

def excel_to_csv(excel_file, output_dir="output_w30"):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Load Excel file
    xls = pd.ExcelFile(excel_file)
    sql_statements = {}

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        
        # Save CSV
        csv_file = os.path.join(output_dir, f"{sheet_name}.csv")
        df.to_csv(csv_file, index=False)
        
# Usage example
excel_file_path = "data v3/Master Data New - Updated.xlsx"
sql_output = excel_to_csv(excel_file_path)

In [2]:
import pandas as pd

# Replace with your Excel file path
excel_file = "data v3/Master Data New - Updated.xlsx"

# Load Excel file
xls = pd.ExcelFile(excel_file)

# Get sheet names
sheet_names = xls.sheet_names

print("Sheet names:", sheet_names)


Sheet names: ['Product', 'Location', 'Customer Source', 'Location Product', 'Location Source', 'Production Source Header', 'Production Source Resource', 'Production Source Item']


# Processing and generation of SQL commands for master table

In [3]:
import pandas as pd
import os
import re

# sanitising column header names
def clean_column_name(col):
    # Remove unwanted characters and replace with underscore
    col = re.sub(r'[\s\-\[\]\(\)\"/%*]', '_', col)
    # Remove multiple underscores and strip leading/trailing ones
    col = re.sub(r'_+', '_', col).strip('_')
    return col

def generate_sql_from_csv(csv_path, table_name=None, output_sql_file="sql_commands_master.sql", schema_name="CURRENT_INVT"):
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # handling the missing/NULL values in tables by replacing them with 0
    df = df.fillna(0)

    
    # Clean column headers
    original_columns = df.columns.tolist()
    cleaned_columns = [clean_column_name(col) for col in original_columns]
    df.columns = cleaned_columns  # Update DataFrame headers

    # Use file name (without extension) as table name if not specified
    if not table_name:
        table_name = os.path.splitext(os.path.basename(csv_path))[0]

    # Start SQL command with schema
    full_table_name = f"{schema_name}.{table_name}"
    sql = f"CREATE TABLE {full_table_name} (\n"
    columns_sql = []

    for col in df.columns:
        # Guess data type
        if pd.api.types.is_integer_dtype(df[col]):
            col_type = "INTEGER"
        elif pd.api.types.is_float_dtype(df[col]):
            col_type = "FLOAT"
        elif pd.api.types.is_bool_dtype(df[col]):
            col_type = "BOOLEAN"
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            col_type = "DATETIME"
        else:
            max_len = df[col].astype(str).map(len).max() or 255
            col_type = f"VARCHAR({min(max_len, 255)})"

        columns_sql.append(f"    {col} {col_type}")

    sql += ",\n".join(columns_sql) + "\n);\n\n"

    # Write to file
    with open(output_sql_file, "a") as f:
        f.write(sql)

    print(f"SQL command for '{full_table_name}' written to '{output_sql_file}'")
    return sql

# Example usage
csvs_name = ['Product', 'Location', 'Customer Source', 'Location Product', 'Location Source', 'Production Source Header', 'Production Source Resource', 'Production Source Item']
for name in csvs_name:
    csv_file_path = f"output_w30/{name}.csv"
    sql_command = generate_sql_from_csv(csv_file_path)


SQL command for 'CURRENT_INVT.Product' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Location' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Customer Source' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Location Product' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Location Source' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Production Source Header' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Production Source Resource' written to 'sql_commands_master.sql'
SQL command for 'CURRENT_INVT.Production Source Item' written to 'sql_commands_master.sql'


In [5]:
from hdbcli import dbapi

# Define schema and table names
schema_name = "CURRENT_INVT"
csvs_name = [
    'Product',
    'Location_Product',
    'Location_Source',
    'Production_Source_Header',
    'Production_Source_Resource',
    'Production_Source_Item'
]

# Database connection parameters — replace with your values
conn_params = {
    "address": "cfe32093-429a-4e59-87dc-9f3e4da891bf.hna2.prod-eu10.hanacloud.ondemand.com",       # e.g., "hostname.com"
    "port": 443,                     # default HANA port
    "user": "DBADMIN",
    "password": "Bcone@1234567"
}

# Generate DROP TABLE statements
drop_statements = [f'DROP TABLE "{schema_name}"."{table_name}";' for table_name in csvs_name]

# Print generated SQL (optional)
for stmt in drop_statements:
    print(stmt)

# Connect to HANA and execute DROP TABLEs
try:
    conn = dbapi.connect(**conn_params)
    cursor = conn.cursor()
    
    for stmt in drop_statements:
        try:
            cursor.execute(stmt)
            print(f"Executed: {stmt}")
        except dbapi.Error as e:
            print(f"Error dropping table: {stmt}\n{e}")

    cursor.close()
    conn.close()
    print("All operations completed.")

except dbapi.Error as conn_err:
    print(f"Connection failed: {conn_err}")


DROP TABLE "CURRENT_INVT"."Product";
DROP TABLE "CURRENT_INVT"."Location_Product";
DROP TABLE "CURRENT_INVT"."Location_Source";
DROP TABLE "CURRENT_INVT"."Production_Source_Header";
DROP TABLE "CURRENT_INVT"."Production_Source_Resource";
DROP TABLE "CURRENT_INVT"."Production_Source_Item";
Connection failed: (-10709, 'Connection failed (RTE:[89013] Socket closed by peer {192.168.0.111:63773 -> 18.156.196.160:443} (192.168.0.111:63773 -> cfe32093-429a-4e59-87dc-9f3e4da891bf.hna2.prod-eu10.hanacloud.ondemand.com:443))')


# transposing the raw transactional data.

this scripts transposes the transcational data. The resulting table has following column header
IDs (product id, location id)
Date (week year combination converted to date)
key figures (e.g. demand, invertory target, project stock for Review DC)

In [31]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os
from pathlib import Path
import numpy as np

def get_week_start_date(week_year_str):
    """
    Convert week-year format (e.g., 'w20 2025') to the first date of that week.
    Assumes ISO week numbering where Monday is the first day of the week.
    """
    if pd.isna(week_year_str):
        return None
        
    week_year_str = str(week_year_str).strip()
    
    # Handle various formats: 'w20 2025', 'W20 2025', 'w20-2025', etc.
    patterns = [
        r'w(\d+)[\s\-_]*(\d{4})',  # w20 2025, w20-2025
        r'(\d{4})[\s\-_]*w(\d+)',  # 2025 w20, 2025-w20
        r'week[\s\-_]*(\d+)[\s\-_]*(\d{4})',  # week 20 2025
        r'(\d{4})[\s\-_]*week[\s\-_]*(\d+)',  # 2025 week 20
    ]
    
    week_num = None
    year = None
    
    for pattern in patterns:
        match = re.search(pattern, week_year_str, re.IGNORECASE)
        if match:
            groups = match.groups()
            # Check which group is the year (4 digits) and which is week
            if len(groups[0]) == 4:  # First group is year
                year = int(groups[0])
                week_num = int(groups[1])
            else:  # Second group is year
                week_num = int(groups[0])
                year = int(groups[1])
            break
    
    if week_num is None or year is None:
        print(f"Could not parse week format: '{week_year_str}'")
        return None
    
    # Validate week number
    if week_num < 1 or week_num > 53:
        print(f"Invalid week number {week_num} in '{week_year_str}'")
        return None
    
    try:
        # Get the first day of the year
        jan_1 = datetime(year, 1, 1)
        
        # Find the first Monday of the year (ISO week 1)
        if jan_1.weekday() <= 3:  # If Jan 1 is Mon-Thu, it's in week 1
            first_monday = jan_1 - timedelta(days=jan_1.weekday())
        else:  # If Jan 1 is Fri-Sun, week 1 starts next Monday
            days_to_monday = 7 - jan_1.weekday()
            first_monday = jan_1 + timedelta(days=days_to_monday)
        
        # Calculate the start date of the target week
        target_date = first_monday + timedelta(weeks=week_num - 1)
        
        return target_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error calculating date for '{week_year_str}': {e}")
        return None

def detect_columns(df):
    """
    Automatically detect ID columns, key figure column, and week columns.
    
    Returns:
        dict: Dictionary with 'id_columns', 'key_figure_column', 'week_columns', 'other_columns'
    """
    columns = df.columns.tolist()
    
    # Detect week columns (columns that match week patterns)
    week_patterns = [
        r'w\d+[\s\-_]*\d{4}',  # w20 2025, w20-2025, w20_2025
        r'week[\s\-_]*\d+[\s\-_]*\d{4}',  # week 20 2025
        r'\d{4}[\s\-_]*w\d+',  # 2025 w20, 2025-w20
    ]
    
    week_columns = []
    for col in columns:
        for pattern in week_patterns:
            if re.search(pattern, str(col), re.IGNORECASE):
                week_columns.append(col)
                break
    
    # Detect key figure column (likely contains categorical data)
    key_figure_candidates = []
    for col in columns:
        if col.lower() in ['key figure', 'key_figure', 'metric', 'measure', 'indicator', 'type']:
            key_figure_candidates.append(col)
    
    # If no obvious key figure column, look for columns with repeated categorical values
    if not key_figure_candidates:
        for col in columns:
            if col not in week_columns:
                unique_ratio = len(df[col].unique()) / len(df)
                if unique_ratio < 0.1 and df[col].dtype == 'object':  # Less than 10% unique values
                    key_figure_candidates.append(col)
    
    key_figure_column = key_figure_candidates[0] if key_figure_candidates else None
    
    # ID columns are remaining non-week, non-key-figure columns, but exclude numeric summary columns
    excluded_columns = week_columns + ([key_figure_column] if key_figure_column else [])
    potential_id_columns = [col for col in columns if col not in excluded_columns]
    
    # Filter out columns that are likely not ID columns (like 'Total', numeric summaries, etc.)
    # Also filter out columns that are mostly NaN
    id_columns = []
    for col in potential_id_columns:
        # Skip columns with names suggesting they're summary/total columns
        if col.lower() in ['total', 'sum', 'average', 'avg', 'count', 'grand total']:
            continue
        
        # Skip columns that are mostly NaN (more than 90% NaN)
        if df[col].isna().sum() / len(df) > 0.9:
            continue
            
        id_columns.append(col)
    
    # Other columns (like 'total' etc.)
    other_columns = [col for col in potential_id_columns if col not in id_columns]
    
    return {
        'id_columns': id_columns,
        'key_figure_column': key_figure_column,
        'week_columns': week_columns,
        'other_columns': other_columns
    }

def transform_single_csv(input_file, output_file, config=None):
    """
    Transform a single CSV from wide format to long format.
    
    Args:
        input_file (str): Path to input CSV file
        output_file (str): Path to output CSV file (optional)
        config (dict): Manual configuration for columns (optional)
    
    Returns:
        pd.DataFrame: Transformed dataframe
    """
    
    print(f"\n{'='*60}")
    print(f"Processing: {input_file}")
    print(f"{'='*60}")
    
    # Read the CSV file
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"Error reading {input_file}: {e}")
        return None
    
    # Clean column names
    df.columns = df.columns.str.strip().str.replace(r"\s+", "_", regex=True)
    # Step 1: Replace string "(None)" with empty string
    df.replace(to_replace=r"\(None\)", value="", regex=True, inplace=True)
    
    # Step 2: Replace blank strings and NaN with 0
    df.replace(to_replace=["", np.nan], value=0, inplace=True)
    
    # Use provided config or detect columns automatically
    if config:
        column_info = config
    else:
        column_info = detect_columns(df)
    
    print(f"Detected columns:")
    print(f"  ID columns: {column_info['id_columns']}")
    print(f"  Key figure column: {column_info['key_figure_column']}")
    print(f"  Week columns: {len(column_info['week_columns'])} columns")
    print(f"  Other columns: {column_info['other_columns']}")
    
    # Show which ID columns have data
    if column_info['id_columns']:
        print(f"  ID column data availability:")
        for col in column_info['id_columns']:
            non_null_pct = (1 - df[col].isna().sum() / len(df)) * 100
            print(f"    {col}: {non_null_pct:.1f}% non-null")
    
    # Show sample key figures
    if column_info['key_figure_column']:
        unique_kf = df[column_info['key_figure_column']].unique()
        print(f"  Key figures ({len(unique_kf)}): {unique_kf[:3]}{'...' if len(unique_kf) > 3 else ''}")
    
    # Validate detection
    if not column_info['week_columns']:
        print("ERROR: No week columns detected!")
        return None
    
    if not column_info['key_figure_column']:
        print("WARNING: No key figure column detected. Proceeding without pivoting key figures.")
        return transform_without_key_figures(df, column_info, output_file)
    
    # Prepare for melting
    id_vars = column_info['id_columns'] + [column_info['key_figure_column']]
    
    # Melt the dataframe
    try:
        melted_df = pd.melt(
            df,
            id_vars=id_vars,
            value_vars=column_info['week_columns'],
            var_name='week_year',
            value_name='value'
        )
    except Exception as e:
        print(f"Error during melting: {e}")
        return None
    
    # Convert week_year to dates
    print("Converting week formats to dates...")
    
    # Debug: Show sample week formats
    sample_weeks = melted_df['week_year'].unique()[:5]
    print(f"Sample week formats: {sample_weeks}")
    
    melted_df['date'] = melted_df['week_year'].apply(get_week_start_date)
    
    # Debug: Show conversion results
    sample_conversions = melted_df[['week_year', 'date']].drop_duplicates().head(5)
    print("Sample conversions:")
    print(sample_conversions.to_string())
    
    # Remove rows where date conversion failed
    before_count = len(melted_df)
    melted_df = melted_df.dropna(subset=['date'])
    after_count = len(melted_df)
    print(f"Rows before date filtering: {before_count}")
    print(f"Rows after date filtering: {after_count}")
    if before_count != after_count:
        print(f"WARNING: {before_count - after_count} rows dropped due to date conversion issues")
        
        # Show failed conversions
        failed_weeks = df.columns[df.columns.str.contains('w', case=False, na=False)][:3]
        print(f"Sample failed week formats: {failed_weeks}")
    
    print("Week year conversion done")
    
    # Debug: Check data before pivoting
    print(f"Data available for pivoting: {len(melted_df)} rows")
    if len(melted_df) > 0:
        print("Sample melted data:")
        print(melted_df.head(3).to_string())
        print(f"Unique key figures: {melted_df[column_info['key_figure_column']].unique()}")
        
        # Check for NaN values in ID columns
        print("Checking ID columns for NaN values:")
        for col in column_info['id_columns']:
            nan_count = melted_df[col].isna().sum()
            print(f"  {col}: {nan_count} NaN values out of {len(melted_df)}")
        
        # If too many NaN values, filter them out or handle differently
        id_cols_with_data = [col for col in column_info['id_columns'] 
                           if melted_df[col].isna().sum() < len(melted_df) * 0.9]
        
        if not id_cols_with_data:
            print("WARNING: All ID columns are mostly NaN. Using index-based grouping.")
            # Add a row identifier to prevent empty pivot
            melted_df['row_id'] = melted_df.index // len(column_info['week_columns'])
            id_cols_with_data = ['row_id']
        
        print(f"Using ID columns for pivot: {id_cols_with_data}")
    else:
        print("ERROR: No data available for pivoting!")
        return None
    
    # Pivot key figures to columns
    try:
        print("key figure cols",column_info['key_figure_column'])
        pivot_df = melted_df.pivot_table(
            index=id_cols_with_data + ['date'],
            columns=column_info['key_figure_column'],
            values='value',
            aggfunc='first'
        ).reset_index()
        
        # Flatten column names
        pivot_df.columns.name = None
        
        print(f"Pivot successful. Shape: {pivot_df.shape}")
        
    except Exception as e:
        print(f"Error during pivoting: {e}")
        print("Attempting alternative approach...")
        
        # Alternative: Don't pivot, just keep as long format
        result_df = melted_df.copy()
        result_df = result_df.sort_values(id_cols_with_data + ['date'])
        return result_df
    
    # Sort the result
    sort_columns = id_cols_with_data + ['date']
    pivot_df = pivot_df.sort_values(sort_columns)
    
    # Print summary
    print(f"\nTransformation Summary:")
    print(f"  Original shape: {df.shape}")
    print(f"  Transformed shape: {pivot_df.shape}")
    print(f"  Key figures found: {[col for col in pivot_df.columns if col not in id_cols_with_data + ['date']]}")
    
    # Save if output file specified
    if output_file:
        try:
            pivot_df.to_csv(output_file, index=False)
            print(f"  Saved to: {output_file}")
        except Exception as e:
            print(f"Error saving file: {e}")
    
    return pivot_df

def transform_without_key_figures(df, column_info, output_file=None):
    """
    Transform CSV when no key figure column is detected (simple unpivot).
    """
    print("Performing simple unpivot transformation...")
    
    melted_df = pd.melt(
        df,
        id_vars=column_info['id_columns'],
        value_vars=column_info['week_columns'],
        var_name='week_year',
        value_name='value'
    )
    
    melted_df['date'] = melted_df['week_year'].apply(get_week_start_date)
    melted_df = melted_df.dropna(subset=['date'])
    melted_df = melted_df.drop('week_year', axis=1)
    
    sort_columns = column_info['id_columns'] + ['date']
    melted_df = melted_df.sort_values(sort_columns)
    
    if output_file:
        melted_df.to_csv(output_file, index=False)
        print(f"Saved to: {output_file}")
    
    return melted_df

def transform_multiple_csv(input_folder, output_folder=None, file_pattern="*.csv"):
    """
    Transform multiple CSV files in a folder.
    
    Args:
        input_folder (str): Path to folder containing input CSV files
        output_folder (str): Path to folder for output files (optional)
        file_pattern (str): Pattern to match files (default: "*.csv")
    
    Returns:
        dict: Dictionary of transformed dataframes
    """
    
    input_path = Path(input_folder)
    if not input_path.exists():
        print(f"Input folder {input_folder} does not exist!")
        return {}
    
    # Create output folder if specified
    if output_folder:
        output_path = Path(output_folder)
        output_path.mkdir(parents=True, exist_ok=True)
    
    # Find all matching files
    csv_files = list(input_path.glob(file_pattern))
    
    if not csv_files:
        print(f"No files found matching pattern {file_pattern} in {input_folder}")
        return {}


def preview_file_structure(input_file, num_rows=3):
    """
    Preview file structure to understand the data before transformation.
    """
    try:
        df = pd.read_csv(input_file)
        df.columns = df.columns.str.strip()
        
        print(f"\n{'='*60}")
        print(f"FILE STRUCTURE PREVIEW: {input_file}")
        print(f"{'='*60}")
        print(f"Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        column_info = detect_columns(df)
        print(f"\nDetected Structure:")
        print(f"  ID columns: {column_info['id_columns']}")
        print(f"  Key figure column: {column_info['key_figure_column']}")
        print(f"  Week columns: {len(column_info['week_columns'])} columns")
        print(f"  Sample week columns: {column_info['week_columns'][:5]}")
        
        if column_info['key_figure_column']:
            print(f"  Unique key figures: {df[column_info['key_figure_column']].unique()}")
        
        print(f"\nFirst {num_rows} rows:")
        print(df.head(num_rows).to_string())
        
    except Exception as e:
        print(f"Error previewing {input_file}: {e}")


# Example usage and configuration
if __name__ == "__main__":
    
    # Option 1: Transform a single file
    # transform_single_csv("output_w29/Demand Fulfillment.csv", "transformed_output_w29/Demand_Fulfillment_ts.csv")
    csvs_name = ['Demand Fulfillment','Review DC', 'Review Component', 'Review Capacity', 'Review Plant', 'Review Vendors','Profit Margin' ]

    for name in csvs_name:
    # Option 1: Transform a single file
        transform_single_csv(f"output_w30/{name}.cszv", f"transformed_w30/{name}_ts.csv")
    #name = 'Profit Margin'
    #transform_single_csv(f"output_w30/{name}.csv", f"ts_w230_dates/{name}.csv")



Processing: output_w30/Demand Fulfillment.csv
Detected columns:
  ID columns: ['Product_ID', 'Customer_ID', 'Customer_Priority']
  Key figure column: Key_Figure
  Week columns: 52 columns
  Other columns: ['Total']
  ID column data availability:
    Product_ID: 100.0% non-null
    Customer_ID: 100.0% non-null
    Customer_Priority: 100.0% non-null
  Key figures (5): ['Consensus Demand' 'Customer Receipts' 'Customer Demand Delivered Late']...
Converting week formats to dates...
Sample week formats: ['W30_2025' 'W31_2025' 'W32_2025' 'W33_2025' 'W34_2025']
Sample conversions:
    week_year        date
0    W30_2025  2025-07-21
35   W31_2025  2025-07-28
70   W32_2025  2025-08-04
105  W33_2025  2025-08-11
140  W34_2025  2025-08-18
Rows before date filtering: 1820
Rows after date filtering: 1820
Week year conversion done
Data available for pivoting: 1820 rows
Sample melted data:
   Product_ID          Customer_ID  Customer_Priority                      Key_Figure week_year  value        dat

In [14]:
import re

def add_schema_to_create_table(input_file, output_file=None, schema='invt_historical_data'):
    with open(input_file, 'r') as f:
        sql = f.read()

    # Add schema before table name in CREATE TABLE statements
    modified_sql = re.sub(
        r'(?i)(CREATE\s+TABLE\s+)(\w+)', 
        rf'\1{schema}.\2', 
        sql
    )

    # Write output
    with open(output_file or input_file, 'w') as f:
        f.write(modified_sql)

    print(f"Updated SQL written to: {output_file or input_file}")

# Example usage
csvs_name = ['Demand Fulfillment_prev','Review DC_prev', 'Review Component_prev', 'Review Capacity_prev', 'Review Plant_prev', 'Review Vendors_prev' ]

for name in csvs_name:
    add_schema_to_create_table(f"previous_files_w29_w30/{name}.csv","sql_commands_history_table.sql")


Updated SQL written to: sql_commands_history_table.sql
Updated SQL written to: sql_commands_history_table.sql
Updated SQL written to: sql_commands_history_table.sql
Updated SQL written to: sql_commands_history_table.sql
Updated SQL written to: sql_commands_history_table.sql
Updated SQL written to: sql_commands_history_table.sql


# updating the above tranformaiton script for adding refreshed date for storing historical data

historical data : schema INVT_HISTORICAL_DATA containing W29 and W30 transaction data. to be referred for scenarios where demand forecasting analysis is needed

<br>
check scenario 11, 12 in the Scenario Analysis Reference sheet

In [27]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os
from pathlib import Path
import numpy as np


def get_week_start_date(week_year_str):
    """
    Convert week-year format (e.g., 'w20 2025') to the first date of that week.
    Assumes ISO week numbering where Monday is the first day of the week.
    """
    if pd.isna(week_year_str):
        return None
        
    week_year_str = str(week_year_str).strip()
    
    # Handle various formats: 'w20 2025', 'W20 2025', 'w20-2025', etc.
    patterns = [
        r'w(\d+)[\s\-_]*(\d{4})',  # w20 2025, w20-2025
        r'(\d{4})[\s\-_]*w(\d+)',  # 2025 w20, 2025-w20
        r'week[\s\-_]*(\d+)[\s\-_]*(\d{4})',  # week 20 2025
        r'(\d{4})[\s\-_]*week[\s\-_]*(\d+)',  # 2025 week 20
    ]
    
    week_num = None
    year = None
    
    for pattern in patterns:
        match = re.search(pattern, week_year_str, re.IGNORECASE)
        if match:
            groups = match.groups()
            # Check which group is the year (4 digits) and which is week
            if len(groups[0]) == 4:  # First group is year
                year = int(groups[0])
                week_num = int(groups[1])
            else:  # Second group is year
                week_num = int(groups[0])
                year = int(groups[1])
            break
    
    if week_num is None or year is None:
        print(f"Could not parse week format: '{week_year_str}'")
        return None
    
    # Validate week number
    if week_num < 1 or week_num > 53:
        print(f"Invalid week number {week_num} in '{week_year_str}'")
        return None
    
    try:
        # Get the first day of the year
        jan_1 = datetime(year, 1, 1)
        
        # Find the first Monday of the year (ISO week 1)
        if jan_1.weekday() <= 3:  # If Jan 1 is Mon-Thu, it's in week 1
            first_monday = jan_1 - timedelta(days=jan_1.weekday())
        else:  # If Jan 1 is Fri-Sun, week 1 starts next Monday
            days_to_monday = 7 - jan_1.weekday()
            first_monday = jan_1 + timedelta(days=days_to_monday)
        
        # Calculate the start date of the target week
        target_date = first_monday + timedelta(weeks=week_num - 1)
        
        return target_date.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"Error calculating date for '{week_year_str}': {e}")
        return None
        
def detect_columns(df):
    """
    Automatically detect ID columns, key figure column, and week columns.
    
    Returns:
        dict: Dictionary with 'id_columns', 'key_figure_column', 'week_columns', 'other_columns'
    """
    columns = df.columns.tolist()
    
    # Detect week columns (columns that match week patterns)
    week_patterns = [
        r'w\d+[\s\-_]*\d{4}',  # w20 2025, w20-2025, w20_2025
        r'week[\s\-_]*\d+[\s\-_]*\d{4}',  # week 20 2025
        r'\d{4}[\s\-_]*w\d+',  # 2025 w20, 2025-w20
    ]
    
    week_columns = []
    for col in columns:
        for pattern in week_patterns:
            if re.search(pattern, str(col), re.IGNORECASE):
                week_columns.append(col)
                break
    
    # Detect key figure column (likely contains categorical data)
    key_figure_candidates = []
    for col in columns:
        if col.lower() in ['key figure', 'key_figure', 'metric', 'measure', 'indicator', 'type']:
            key_figure_candidates.append(col)
    
    # If no obvious key figure column, look for columns with repeated categorical values
    if not key_figure_candidates:
        for col in columns:
            if col not in week_columns:
                unique_ratio = len(df[col].unique()) / len(df)
                if unique_ratio < 0.1 and df[col].dtype == 'object':  # Less than 10% unique values
                    key_figure_candidates.append(col)
    
    key_figure_column = key_figure_candidates[0] if key_figure_candidates else None
    
    # ID columns are remaining non-week, non-key-figure columns, but exclude numeric summary columns
    excluded_columns = week_columns + ([key_figure_column] if key_figure_column else [])
    potential_id_columns = [col for col in columns if col not in excluded_columns]
    
    # Filter out columns that are likely not ID columns (like 'Total', numeric summaries, etc.)
    # Also filter out columns that are mostly NaN
    id_columns = []
    for col in potential_id_columns:
        # Skip columns with names suggesting they're summary/total columns
        if col.lower() in ['total', 'sum', 'average', 'avg', 'count', 'grand total']:
            continue
        
        # Skip columns that are mostly NaN (more than 90% NaN)
        if df[col].isna().sum() / len(df) > 0.9:
            continue
            
        id_columns.append(col)
    
    # Other columns (like 'total' etc.)
    other_columns = [col for col in potential_id_columns if col not in id_columns]
    
    return {
        'id_columns': id_columns,
        'key_figure_column': key_figure_column,
        'week_columns': week_columns,
        'other_columns': other_columns
    }

def transform_single_csv(input_file, output_file,current_date_str, config=None):
    """
    Transform a single CSV from wide format to long format.
    
    Args:
        input_file (str): Path to input CSV file
        output_file (str): Path to output CSV file (optional)
        config (dict): Manual configuration for columns (optional)
    
    Returns:
        pd.DataFrame: Transformed dataframe
    """
    
    print(f"\n{'='*60}")
    print(f"Processing: {input_file}")
    print(f"{'='*60}")
    
    # Read the CSV file
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"Error reading {input_file}: {e}")
        return None
    
    # Clean column names
    df.columns = df.columns.str.strip().str.replace(r"\s+", "_", regex=True)
    # Step 1: Replace string "(None)" with empty string
    df.replace(to_replace=r"\(None\)", value="", regex=True, inplace=True)
    
    # Step 2: Replace blank strings and NaN with 0
    df.replace(to_replace=["", np.nan], value=0, inplace=True)
    
    # Use provided config or detect columns automatically
    if config:
        column_info = config
    else:
        column_info = detect_columns(df)
    
    print(f"Detected columns:")
    print(f"  ID columns: {column_info['id_columns']}")
    print(f"  Key figure column: {column_info['key_figure_column']}")
    print(f"  Week columns: {len(column_info['week_columns'])} columns")
    print(f"  Other columns: {column_info['other_columns']}")
    
    # Show which ID columns have data
    if column_info['id_columns']:
        print(f"  ID column data availability:")
        for col in column_info['id_columns']:
            non_null_pct = (1 - df[col].isna().sum() / len(df)) * 100
            print(f"    {col}: {non_null_pct:.1f}% non-null")
    
    # Show sample key figures
    if column_info['key_figure_column']:
        unique_kf = df[column_info['key_figure_column']].unique()
        print(f"  Key figures ({len(unique_kf)}): {unique_kf[:3]}{'...' if len(unique_kf) > 3 else ''}")
    
    # Validate detection
    if not column_info['week_columns']:
        print("ERROR: No week columns detected!")
        return None
    
    # Prepare for melting
    id_vars = column_info['id_columns'] + [column_info['key_figure_column']]
    
    # Melt the dataframe
    try:
        melted_df = pd.melt(
            df,
            id_vars=id_vars,
            value_vars=column_info['week_columns'],
            var_name='week_year',
            value_name='value'
        )
    except Exception as e:
        print(f"Error during melting: {e}")
        return None
    
    # Convert week_year to dates
    print("Converting week formats to dates...")
    
    # Debug: Show sample week formats
    sample_weeks = melted_df['week_year'].unique()[:5]
    print(f"Sample week formats: {sample_weeks}")
    
    melted_df['date'] = melted_df['week_year'].apply(get_week_start_date)
    
    # Remove rows where date conversion failed
    before_count = len(melted_df)
    after_count = len(melted_df)
    print(f"Rows before date filtering: {before_count}")
    print(f"Rows after date filtering: {after_count}")
    if before_count != after_count:
        print(f"WARNING: {before_count - after_count} rows dropped due to date conversion issues")
        
        # Show failed conversions
        failed_weeks = df.columns[df.columns.str.contains('w', case=False, na=False)][:3]
        print(f"Sample failed week formats: {failed_weeks}")
    
    print("Week year conversion done")
    
    # Debug: Check data before pivoting
    print(f"Data available for pivoting: {len(melted_df)} rows")
    if len(melted_df) > 0:
        print("Sample melted data:")
        print(melted_df.head(3).to_string())
        print(f"Unique key figures: {melted_df[column_info['key_figure_column']].unique()}")
        
        # Check for NaN values in ID columns
        print("Checking ID columns for NaN values:")
        for col in column_info['id_columns']:
            nan_count = melted_df[col].isna().sum()
            print(f"  {col}: {nan_count} NaN values out of {len(melted_df)}")
        
        # If too many NaN values, filter them out or handle differently
        id_cols_with_data = [col for col in column_info['id_columns'] 
                           if melted_df[col].isna().sum() < len(melted_df) * 0.9]
        
        if not id_cols_with_data:
            print("WARNING: All ID columns are mostly NaN. Using index-based grouping.")
            # Add a row identifier to prevent empty pivot
            melted_df['row_id'] = melted_df.index // len(column_info['week_columns'])
            id_cols_with_data = ['row_id']
        
        print(f"Using ID columns for pivot: {id_cols_with_data}")
    else:
        print("ERROR: No data available for pivoting!")
        return None
    
    # Pivot key figures to columns
    try:
        print("key figure cols",column_info['key_figure_column'])
        pivot_df = melted_df.pivot_table(
            index=id_cols_with_data + ['week_year'],
            columns=column_info['key_figure_column'],
            values='value',
            aggfunc='first'
        ).reset_index()
        
        # Flatten column names
        pivot_df.columns.name = None
        
        print(f"Pivot successful. Shape: {pivot_df.shape}")
        
    except Exception as e:
        print(f"Error during pivoting: {e}")
        print("Attempting alternative approach...")
        
        # Alternative: Don't pivot, just keep as long format
        result_df = melted_df.copy()
        result_df = result_df.sort_values(id_cols_with_data + ['week_year'])
        return result_df
    
    # Sort the result
    sort_columns = id_cols_with_data + ['week_year']
    pivot_df = pivot_df.sort_values(sort_columns)

    # ✅ Add refresh_date column with current date
    #current_date_str = datetime.today().strftime('%Y-%m-%d')
    #current_date_str = '21-07-2025'
    #pivot_df['refresh_date'] = current_date_str
    
    # Print summary
    print(f"\nTransformation Summary:")
    print(f"  Original shape: {df.shape}")
    print(f"  Transformed shape: {pivot_df.shape}")
    print(f"  Key figures found: {[col for col in pivot_df.columns if col not in id_cols_with_data + ['week_year']]}")
    
    # Save if output file specified
    if output_file:
        try:
            # Append to existing CSV if it exists, else create new one
            if os.path.exists(output_file):
                existing_df = pd.read_csv(output_file)
                combined_df = pd.concat([existing_df, pivot_df], ignore_index=True)
                combined_df.to_csv(output_file, index=False)
                print(f"  Appended to existing CSV: {output_file}")
            else:
                pivot_df.to_csv(output_file, index=False)
                print(f"  Created new CSV: {output_file}")

        except Exception as e:
            print(f"Error saving/appending to CSV file: {e}")

    # if output_file:
    #     try:
    #         pivot_df.to_csv(output_file, index=False)
    #         print(f"  Saved to: {output_file}")
    #     except Exception as e:
    #         print(f"Error saving file: {e}")
    
    return pivot_df






def preview_file_structure(input_file, num_rows=3):
    """
    Preview file structure to understand the data before transformation.
    """
    try:
        df = pd.read_csv(input_file)
        df.columns = df.columns.str.strip()
        
        # print(f"\n{'='*60}")
        # print(f"FILE STRUCTURE PREVIEW: {input_file}")
        # print(f"{'='*60}")
        # print(f"Shape: {df.shape}")
        # print(f"Columns: {list(df.columns)}")
        
        # column_info = detect_columns(df)
        # print(f"\nDetected Structure:")
        # print(f"  ID columns: {column_info['id_columns']}")
        # print(f"  Key figure column: {column_info['key_figure_column']}")
        # print(f"  Week columns: {len(column_info['week_columns'])} columns")
        # print(f"  Sample week columns: {column_info['week_columns'][:5]}")
        
        # if column_info['key_figure_column']:
        #     print(f"  Unique key figures: {df[column_info['key_figure_column']].unique()}")
        
        # print(f"\nFirst {num_rows} rows:")
        # print(df.head(num_rows).to_string())

        print(f'{input_file} is transformed and saved')
    except Exception as e:
        print(f"Error previewing {input_file}: {e}")


# Example usage and configuration
if __name__ == "__main__":

    # csvs_name = ['Demand Fulfillment','Review DC', 'Review Component', 'Review Capacity', 'Review Plant', 'Review Vendors','Profit Margin' ]

    # for name in csvs_name:
    # # Option 1: Transform a single file
    #     transform_single_csv(f"output_w30/{name}.csv", f"transformed_w30/{name}_ts.csv",'21-07-2025')
    name = 'Profit Margin'
    transform_single_csv(f"output_w30/{name}.csv", f"previous_files_w29_w30/{name}_prev_test.csv",'21-07-2025')


Processing: output_w30/Profit Margin.csv
Detected columns:
  ID columns: ['Product_ID', 'Customer_ID', 'Unnamed:_3']
  Key figure column: Key_Figure
  Week columns: 52 columns
  Other columns: ['Total']
  ID column data availability:
    Product_ID: 100.0% non-null
    Customer_ID: 100.0% non-null
    Unnamed:_3: 100.0% non-null
  Key figures (5): ['Customer Receipts' 'Planned Price' 'Constrained Demand Rev.']...
Converting week formats to dates...
Sample week formats: ['W30_2025' 'W31_2025' 'W32_2025' 'W33_2025' 'W34_2025']
Rows before date filtering: 1820
Rows after date filtering: 1820
Week year conversion done
Data available for pivoting: 1820 rows
Sample melted data:
   Product_ID Customer_ID  Unnamed:_3               Key_Figure week_year  value        date
0  FG-100-001   CUST-1000         0.0        Customer Receipts  W30_2025  250.0  2025-07-21
1  FG-100-001   CUST-1000         0.0            Planned Price  W30_2025    0.0  2025-07-21
2  FG-100-001   CUST-1000         0.0  Con